In [41]:
from keras.layers import Input, Dense, BatchNormalization, Reshape, Flatten, Conv2D, LeakyReLU, UpSampling2D
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Dropout
import tensorflow as tf
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Input, Dense, Lambda, Conv2D, Flatten, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization, UpSampling2D
from keras.models import Model
from keras import backend as K


In [42]:
latent_dim = 8

# Кодировщик

In [43]:
def build_encoder(input_shape, latent_dim):
    inputs = Input(shape=input_shape, name='encoder_input')
    x = Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = Conv2D(256, 3, padding='same', activation='relu', strides=(2, 2))(x)
    x = Flatten()(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    return Model(inputs, [z_mean, z_log_var, z], name='encoder')


# Декодер

In [44]:
def build_decoder(latent_dim):
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(7*7*64, activation='relu')(latent_inputs)
    x = Reshape((7, 7, 64))(x)
    x = Conv2DTranspose(128, 3, padding='same', activation='relu', strides=(2, 2))(x)
    x = Conv2DTranspose(64, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
    outputs = Conv2DTranspose(1, 3, padding='same', activation='sigmoid')(x)
    return Model(latent_inputs, outputs, name='decoder')



# Функция выборки

In [45]:
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


# VAE

In [46]:
def build_vae(encoder, decoder):
    inputs = Input(shape=input_shape, name='vae_input')
    z_mean, z_log_var, z = encoder(inputs)
    reconstructed = decoder(z)
    vae = Model(inputs, reconstructed, name='vae')
    return vae

# GAN

# Дискриминатор

In [47]:
def build_discriminator(input_shape):
    img_input = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(img_input)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU()(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(img_input, x)

discriminator = build_discriminator(input_shape=(28, 28, 1))
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_24 (LeakyReLU)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_25 (LeakyReLU)      │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,769 (315.50 KB)

 Trainable params: 80,769 (315.50 KB)

 Non-trainable params: 0 (0.00 B)

# Генератор

In [48]:
def build_generator(latent_dim):
    z_input = Input(shape=(latent_dim,))
    x = Dense(7*7*128)(z_input)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Reshape((7, 7, 128))(x)
    x = UpSampling2D()(x)
    x = Conv2D(128, kernel_size=3, padding='same')(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = UpSampling2D()(x)
    x = Conv2D(64, kernel_size=3, padding='same')(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Conv2D(1, kernel_size=3, activation='sigmoid', padding='same')(x)
    return Model(z_input, x)

generator = build_generator(latent_dim=latent_dim)
generator.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 6272)           │        56,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_26 (LeakyReLU)      │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 6272)           │        25,088 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_7 (Reshape)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_8 (UpSampling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_27 (LeakyReLU)      │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_9 (UpSampling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 28, 28, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_28 (LeakyReLU)      │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 28, 28, 1)      │           577 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 304,257 (1.16 MB)

 Trainable params: 291,329 (1.11 MB)

 Non-trainable params: 12,928 (50.50 KB)

# Объединение VAE и GAN

In [49]:
class VAEGAN(Model):
    def __init__(self, vae, generator, discriminator, **kwargs):
        super(VAEGAN, self).__init__(**kwargs)
        self.vae = vae
        self.generator = generator
        self.discriminator = discriminator

    def compile(self, vae_optimizer, gan_optimizer, vae_loss_fn, gan_loss_fn):
        super(VAEGAN, self).compile()
        self.vae_optimizer = vae_optimizer
        self.gan_optimizer = gan_optimizer
        self.vae_loss_fn = vae_loss_fn
        self.gan_loss_fn = gan_loss_fn

    def train_step(self, data):
        real_images, labels = data[0], data[1]
        # VAE training
        with tf.GradientTape() as tape:
            reconstructed_images = self.vae(real_images)
            vae_loss = self.vae_loss_fn(real_images, reconstructed_images) + sum(self.vae.losses)
        grads = tape.gradient(vae_loss, self.vae.trainable_weights)
        self.vae_optimizer.apply_gradients(zip(grads, self.vae.trainable_weights))

        # GAN training
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
        generated_images = self.generator(random_latent_vectors)
        combined_images = tf.concat([generated_images, real_images], axis=0)
        labels = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, 1))], axis=0)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            gan_loss = self.gan_loss_fn(labels, predictions)
        grads = tape.gradient(gan_loss, self.discriminator.trainable_weights)
        self.gan_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        misleading_labels = tf.ones((batch_size, 1))
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            gan_loss = self.gan_loss_fn(misleading_labels, predictions)
        grads = tape.gradient(gan_loss, self.generator.trainable_weights)
        self.gan_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"vae_loss": vae_loss, "gan_loss": gan_loss}

vae_gan = VAEGAN(vae, generator, discriminator)
vae_gan.compile(
    vae_optimizer=tf.keras.optimizers.Adam(),
    gan_optimizer=tf.keras.optimizers.Adam(),
    vae_loss_fn=tf.keras.losses.BinaryCrossentropy(),
    gan_loss_fn=tf.keras.losses.BinaryCrossentropy()
)


# Обучение модели

In [50]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Параметры модели
input_shape = (28, 28, 1)
latent_dim = 8

# Построение модели
encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(latent_dim)
vae = build_vae(encoder, decoder)

# Компиляция модели
vae.compile(optimizer='adam', loss='binary_crossentropy')

# Обучение модели
vae.fit(x_train, x_train, epochs=10, batch_size=128, validation_data=(x_test, x_test))


Epoch 1/10


AttributeError: Exception encountered when calling Lambda.call().

[1mmodule 'keras.api.backend' has no attribute 'shape'[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 8), dtype=float32)', 'tf.Tensor(shape=(None, 8), dtype=float32)']
  • mask=['None', 'None']
  • training=True